# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [0]:
#!git clone https://github.com/tensorflow/models.git
#!apt-get install protobuf-compiler python-pil python-lxml python-tk
# import os
# os.chdir('models/research/')
#!protoc object_detection/protos/*.proto --python_out=.


In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf

import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  print(tf.__version__)
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

/home/uniquetrij/Projects/SecureIt/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/local/anaconda3/envs/SecureIt/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/anaconda3/envs/SecureIt/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/anaconda3/envs/SecureIt/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/anaconda3/envs/SecureIt/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/anaconda3/envs/SecureIt/lib/

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_PATH = "object_detection/pretrained/"
#MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'  # SSD Mobilenet
MODEL_NAME = 'faster_rcnn_inception_v2_coco_2018_01_28' # Faster RCNN 

MODEL_FILE = MODEL_PATH + MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_PATH + MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('object_detection/data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

HTTPError: HTTP Error 403: Forbidden

## Load a (frozen) Tensorflow model into memory.

In [12]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [13]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [14]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [15]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'object_detection/test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 1000) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [16]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [1]:
for image_path in TEST_IMAGE_PATHS:
  try:
    image = Image.open(image_path)
  except:
    continue
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

NameError: name 'TEST_IMAGE_PATHS' is not defined

In [0]:
def visualiz(img, detection_boxes, detection_classes, detection_scores):
  height, width = img.shape[0], img.shape[1]
  region = [[264., 355.],
      [369., 202.],
      [600., 202.],
      [600., 355.]]
  bbPath = mplPath.Path(
      np.array(region))
  red = (255, 0, 0)
  green = (0, 255, 0)  
  for i in range(len(detection_boxes)):
#       print(i)
    if detection_classes[i] == 1 and detection_scores[i] > 0.3 :
      y_tl, x_tl, y_br, x_br = detection_boxes[i]*[height, width, height, width]
      path = [[x_tl, y_tl],
             [x_tl, y_br],
             [x_br, y_br],
             [x_br, y_tl]]
      
      color = green      
      if bbPath.contains_point(((x_br+x_tl)/2, y_br)):
        color = red
      
      img = cv2.polylines(img, [np.int32(path)], 1, color, 3)
#   plt.figure(figsize=IMAGE_SIZE)
#   plt.imshow(img)
#   plt.show()
  return img

In [0]:
import matplotlib.path as mplPath
import numpy as np

for image_path in TEST_IMAGE_PATHS:
  #image = Image.open(image_path)
  import cv2
  image = cv2.imread(image_path)
#   print(type(image))
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  if image is None:
    continue
  height, width = image.shape[0], image.shape[1]
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image, detection_graph)
  # Visualization of the results of a detection.
  
  visualiz(image,output_dict['detection_boxes'],output_dict['detection_classes'],output_dict['detection_scores'])

In [0]:
import os 
os.chdir('object_detection/test_images/')

In [54]:
# from google.colab import files
# 
# uploaded = files.upload()
# 
# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

Saving VID-20180801-WA0004.mp4 to VID-20180801-WA0004.mp4
User uploaded file "VID-20180801-WA0004.mp4" with length 3873569 bytes


In [0]:
os.chdir('../..')

In [0]:
#!pip install moviepy

In [0]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
def process_image(image):
    height, width = image.shape[0], image.shape[1]
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image, axis=0)
  # Actual detection.
    output_dict = run_inference_for_single_image(image, detection_graph)
#     print(output_dict['detection_boxes'])
    image = visualiz(image,output_dict['detection_boxes'],output_dict['detection_classes'],output_dict['detection_scores'])
    
    return image
            

In [0]:
white_output = 'object_detection/test_images/output.mp4'
clip1 = VideoFileClip("object_detection/test_images/VID-20180801-WA0004.mp4")#.subclip(0,2)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!s
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video object_detection/test_images/output.mp4
[MoviePy] Writing video object_detection/test_images/output.mp4



  8%|▊         | 35/465 [02:22<29:12,  4.08s/it]


 15%|█▌        | 70/465 [04:48<27:08,  4.12s/it]


 23%|██▎       | 105/465 [07:12<24:42,  4.12s/it]


 24%|██▍       | 111/465 [07:56<25:18,  4.29s/it]

In [71]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [0]:
# from google.colab import files
# 
# 
# files.download('object_detection/test_images/output.mp4')